## Sprint22 深層学習スクラッチ リカレントニューラルネットワーク

#### 1.このSprintについて

##### Sprintの目的
スクラッチを通してリカレントニューラルネットワークの基礎を理解する

##### どのように学ぶか
スクラッチでリカレントニューラルネットワークの実装を行います。

#### 2.リカレントニューラルネットワークスクラッチ
リカレントニューラルネットワーク（RNN） のクラスをスクラッチで作成していきます。NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。

フォワードプロパゲーションの実装を必須課題とし、バックプロパゲーションの実装はアドバンス課題とします。

クラスの名前はScratchSimpleRNNClassifierとしてください。クラスの構造などは以前のSprintで作成したScratchDeepNeuralNetrowkClassifierを参考にしてください。

### 【問題1】SimpleRNNのフォワードプロパゲーション実装 
SimpleRNNのクラスSimpleRNNを作成してください。基本構造はFCクラスと同じになります。  
フォワードプロパゲーションの数式は以下のようになります。ndarrayのshapeがどうなるかを併記しています。  
バッチサイズをbatch_size、入力の特徴量数をn_features、RNNのノード数をn_nodesとして表記します。活性化関数はtanhとして進めますが、これまでのニューラルネットワーク同様にReLUなどに置き換えられます。

$$a_t = x_t \cdot W_x + h_{t-1} \cdot W_h + B$$$$h_t = tanh(a_t)$$

$a_t$ : 時刻tの活性化関数を通す前の状態 (batch_size, n_nodes)

$h_t$ : 時刻tの状態・出力 (batch_size, n_nodes)

$x_t$ : 時刻tの入力 (batch_size, n_features)

$W_x$ : 入力に対する重み (n_features, n_nodes)

$h_{t−1}$ : 時刻t-1の状態（前の時刻から伝わる順伝播） (batch_size, n_nodes)

$W_h$ : 状態に対する重み。 (n_nodes, n_nodes)

$B$ : バイアス項 (n_nodes,)

初期状態 $h_0$ は全て0とすることが多いですが、任意の値を与えることも可能です。

上記の処理を系列数n_sequences回繰り返すことになります。RNN全体への入力 $x$ は(batch_size, n_sequences, n_features)のような配列で渡されることになり、そこから各時刻の配列を取り出していきます。

分類問題であれば、それぞれの時刻のhに対して全結合層とソフトマックス関数（またはシグモイド関数）を使用します。タスクによっては最後の時刻のhだけを使用することもあります。

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import math
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [2]:
class ScratchSimpleRNNClassifier():
    """
    シンプルな三層ニューラルネットワーク分類器

    Parameters
    ----------
    bp : int
        バックプロパゲーション回数
    Attributes
    ----------
    """
    def __init__(self):
        pass

    def fit(self, x, w_x, w_h):
        """
        ニューラルネットワーク分類器を学習する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練用データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練用データの正解値
        X_val : 次の形のndarray, shape (n_samples, n_features)
            検証用データの特徴量
        y_val : 次の形のndarray, shape (n_samples, )
            検証用データの正解値
        """
        self.batch_size = x.shape[0] # 1
        self.n_sequences = x.shape[1] # 3
        self.n_features = x.shape[2] # 2
        self.n_nodes = w_x.shape[1] # 4
        h = np.zeros((self.batch_size, self.n_nodes)) # (batch_size, n_nodes)
        
        for c in range(self.batch_size):
            for i in range(self.n_sequences):
                if i==0:
                    a = np.dot(x[c,i,:],w_x) + np.dot(h, w_h) + b
                    a=np.tanh(a)
                else:
                    a = np.dot(x[c,i,:],w_x) + np.dot(a, w_h) + b
                    a=np.tanh(a)
        return  a

### 【問題2】小さな配列でのフォワードプロパゲーションの実験 
小さな配列でフォワードプロパゲーションを考えてみます。
入力x、初期状態h、重みw_xとw_h、バイアスbを次のようにします。
ここで配列xの軸はバッチサイズ、系列数、特徴量数の順番です。

In [3]:
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes)) # (batch_size, n_nodes)
b = np.array([1, 1, 1, 1]) # (n_nodes,)

In [4]:
display(x)
display(w_x.shape)
display(w_h.shape)
display(h.shape)
display(n_sequences)

array([[[0.01, 0.02],
        [0.02, 0.03],
        [0.03, 0.04]]])

(2, 4)

(4, 4)

(1, 4)

3

In [5]:
#x.reshape(x.shape[1],x.shape[2])
a=np.dot(x[0,:],w_x) + np.dot(h, w_h) + b
a=np.tanh(a)
a

array([[0.76188798, 0.76213958, 0.76239095, 0.76255841],
       [0.76205574, 0.76247469, 0.76289301, 0.76318545],
       [0.7622234 , 0.76280939, 0.76339414, 0.76381105]])

In [6]:
for c in range(batch_size):
    for i in range(n_sequences):
        if i==0:
            a = np.dot(x[c,i,:],w_x) + np.dot(h, w_h) + b
            a=np.tanh(a)
        else:
            a = np.dot(x[c,i,:],w_x) + np.dot(a, w_h) + b
            a=np.tanh(a)
display(a)

array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]])

In [8]:
src_RNN=ScratchSimpleRNNClassifier()
A=src_RNN.fit(x, w_x, w_h)
A

array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]])

In [ ]:
#答え
h = np.array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]]) # (batch_size, n_nodes)